# [1] data merge

## (1) customer_id, order_id 기준으로 병합
 > 고유값을 key로 merge 시행  
 > 각 주문별 배송일자가 다를 수 있음을 고려하여 중복 허용  
 > 필요한 정보값이 가장 많은 'order' 데이터셋을 기반으로 merge 시행

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

!pip install koreanize-matplotlib
import koreanize_matplotlib

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 55.1 MB/s eta 0:00:00


# 데이터 전처리

In [ ]:
# CSV 파일 읽어오기

customers = pd.read_csv('/content/drive/MyDrive/새싹캠퍼스/데이터 분석/팀프로젝트1/olist_customers_dataset.csv')
items = pd.read_csv('/content/drive/MyDrive/새싹캠퍼스/데이터 분석/팀프로젝트1/olist_order_items_dataset.csv')
payments = pd.read_csv('/content/drive/MyDrive/새싹캠퍼스/데이터 분석/팀프로젝트1/olist_order_payments_dataset.csv')
orders = pd.read_csv('/content/drive/MyDrive/새싹캠퍼스/데이터 분석/팀프로젝트1/olist_orders_dataset.csv')
products = pd.read_csv('/content/drive/MyDrive/새싹캠퍼스/데이터 분석/팀프로젝트1/olist_products_dataset.csv')
reviews = pd.read_csv('/content/drive/MyDrive/새싹캠퍼스/데이터 분석/팀프로젝트1/olist_order_reviews_dataset.csv')

## 데이터 병합 : orders를 기준으로

In [ ]:
# 배송 지연 여부 변수 생성
orders['is_delayed'] = orders['order_delivered_customer_date'] > orders['order_estimated_delivery_date']

In [ ]:
# 병합: orders + customers
raw = orders.merge(customers, on='customer_id', how='left')

# 병합: + payments
raw = raw.merge(payments, on='order_id', how='left')

# 병합: + items
raw = raw.merge(items, on='order_id', how='left')

# 병합: + products
raw = raw.merge(products, on='product_id', how='left')

# 병합: + reviews
raw = raw.merge(reviews, on='order_id', how='left')


raw.shape
raw.info()

(119143, 37)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119143 entries, 0 to 119142
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       119143 non-null  object 
 1   customer_id                    119143 non-null  object 
 2   order_status                   119143 non-null  object 
 3   order_purchase_timestamp       119143 non-null  object 
 4   order_approved_at              118966 non-null  object 
 5   order_delivered_carrier_date   117057 non-null  object 
 6   order_delivered_customer_date  115722 non-null  object 
 7   order_estimated_delivery_date  119143 non-null  object 
 8   is_delayed                     119143 non-null  bool   
 9   customer_unique_id             119143 non-null  object 
 10  customer_zip_code_prefix       119143 non-null  int64  
 11  customer_city                  119143 non-null  object 
 12  customer_state                

In [ ]:
# 시간 데이터 datetime 형식으로 변환
raw['order_delivered_customer_date'] = pd.to_datetime(raw['order_delivered_customer_date'])
raw['order_estimated_delivery_date'] = pd.to_datetime(raw['order_estimated_delivery_date'])
raw['order_delivered_carrier_date'] = pd.to_datetime(raw['order_delivered_carrier_date'])
raw['order_approved_at'] = pd.to_datetime(raw['order_approved_at'])
raw['order_purchase_timestamp'] = pd.to_datetime(raw['order_purchase_timestamp'])
raw['shipping_limit_date'] = pd.to_datetime(raw['shipping_limit_date'])

# 배송 지연 여부 변수 생성
raw['is_delayed'] = (raw['order_delivered_customer_date'] > raw['order_estimated_delivery_date']).astype(int)

# 배송 지연 일수 계산
raw['delay_days'] = (raw['order_delivered_customer_date'] - raw['order_estimated_delivery_date']).dt.days

# 음수(예정보다 빨리 도착한 경우)는 0으로 대체
raw['delay_days'] = raw['delay_days'].apply(lambda x: x if x > 0 else 0)

# 중복행 확인
raw.duplicated().sum()

# 결측치 확인
raw.isnull().sum()[raw.isnull().sum()>0] 